## Collaborative Filtering Example

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

### Setting sample data

In [2]:
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 3],
    'title': ['The Matrix', 'Titanic', 'The Avengers', 'Titanic', 'Shrek', 'The Matrix', 'Shrek', 'Toy Story', 'Avatar'],
    'rating': [5, 4, 5, 5, 3, 5, 4, 5, 4]
}

df = pd.DataFrame(data)

### Dataset example

In [3]:
df.head()

,user_id,title,rating
0,1,The Matrix,5
1,1,Titanic,4
2,1,The Avengers,5
3,2,Titanic,5
4,2,Shrek,3


### User-Item Interaction Matrix
*  pivot_table: Converte o DataFrame em uma matriz onde as linhas são user_id, as colunas são title e os valores são rating

In [5]:
user_item_matrix = df.pivot_table(index='user_id', columns='title', values='rating').fillna(0)

print("\nUser-Item Interaction Matrix:")
user_item_matrix


User-Item Interaction Matrix:


title,Avatar,Shrek,The Avengers,The Matrix,Titanic,Toy Story
user_id,,,,,,
1,0.0,0.0,5.0,5.0,4.0,0.0
2,0.0,3.0,0.0,0.0,5.0,0.0
3,4.0,4.0,0.0,5.0,0.0,5.0


### Computing cosene similatiry
* Calcula a similaridade coseno entre todos os pares de usuários

In [6]:
user_item_sparse_matrix = csr_matrix(user_item_matrix.values)

# Computar a matriz de similaridade coseno
user_similarity = cosine_similarity(user_item_sparse_matrix)

print("\nUser Similarity Matrix:")
user_similarity


User Similarity Matrix:


array([[1.        , 0.42220033, 0.33982953],
       [0.42220033, 1.        , 0.22726621],
       [0.33982953, 0.22726621, 1.        ]])

### Get Recommendations

In [7]:
def get_collaborative_recommendations(user_id, user_item_matrix, user_similarity):
    user_idx = user_id - 1  # Índice do usuário
    
    # pontuações de similaridade para o usuário
    sim_scores = user_similarity[user_idx]
    
    # avaliações do usuário
    user_ratings = user_item_matrix.iloc[user_idx]
    
    weighted_ratings = pd.DataFrame(index=user_item_matrix.columns, columns=['rating'])
    weighted_ratings['rating'] = 0
    
    # Computar a soma ponderada das avaliações
    for i in range(len(sim_scores)):
        if i != user_idx:
            weighted_ratings['rating'] += sim_scores[i] * user_item_matrix.iloc[i]
    
    # Normalizar as avaliações ponderadas pela soma das similaridades
    weighted_ratings['rating'] /= sum(sim_scores) - 1
    
    # Remover filmes que o usuário já avaliou
    recommendations = weighted_ratings[~user_ratings.astype(bool)]
    
    # Ordenar as recomendações por avaliação
    recommendations = recommendations.sort_values(by='rating', ascending=False)
    
    # Retornar as 5 principais recomendações
    return recommendations.head(5)


In [8]:
# Exemplo: Obter recomendações para o usuário 1
print("\nRecomendações para o Usuário 1:")
print(get_collaborative_recommendations(1, user_item_matrix, user_similarity))

# Exemplo: Obter recomendações para o usuário 2
print("\nRecomendações para o Usuário 2:")
print(get_collaborative_recommendations(2, user_item_matrix, user_similarity))

# Exemplo: Obter recomendações para o usuário 3
print("\nRecomendações para o Usuário 3:")
print(get_collaborative_recommendations(3, user_item_matrix, user_similarity))



Recomendações para o Usuário 1:
             rating
title              
Shrek      3.445953
Toy Story  2.229765
Avatar     1.783812

Recomendações para o Usuário 2:
                rating
title                 
The Matrix    5.000000
The Avengers  3.250362
Toy Story     1.749638
Avatar        1.399710

Recomendações para o Usuário 3:
                rating
title                 
Titanic       4.400755
The Avengers  2.996227
